# Currency Futures Strategy Backtest Notebook

This notebook loads local daily OHLC files and runs backtests for each symbol/date/strategy combo.

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import json
from pathlib import Path
from currency_strategy_backtester import (
    load_strategies,
    fetch_price_data,
    simulate_trade,
    resolve_strategy_name
)

In [ ]:
# Load trade signals
signals_df = pd.read_csv("trade_signals.csv")
signals_df.head()


In [ ]:
# Load strategy definitions
strategies = load_strategies("strategies_complete.json")
list(strategies.keys())


In [ ]:
# Backtest using local CSVs from ./data/
results = []

for _, row in signals_df.iterrows():
    symbol = row["symbol"]
    raw_name = row["strategy"]
    try:
        signal_date = pd.to_datetime(row["date"], format='%m/%d/%y')
    except ValueError:
        print(f"❌ Invalid date format: {row['date']}")
        continue

    alternate_symbols = {"AD": "6A", "BP": "6B", "CD": "6C", "EC": "6E", "SF": "6S"}
    if symbol in alternate_symbols:
        symbol = alternate_symbols[symbol]

    resolved_name = resolve_strategy_name(raw_name, list(strategies.keys()))
    if not resolved_name:
        print(f"⚠️ Could not resolve strategy name: '{raw_name}'")
        continue

    # Attempt to load from ./data/ folder
    local_path = Path(f"data/{symbol}.csv")
    if not local_path.exists():
        print(f"⚠️ No local CSV found for {symbol}")
        continue

    try:
        df = pd.read_csv(local_path)
        df["Date"] = pd.to_datetime(df["Date"], format="%Y%m%d")
        df.set_index("Date", inplace=True)
        df = df[["Open", "High", "Low", "Close", "Volume"]]
        df = df.loc[(df.index >= signal_date - timedelta(days=10)) & (df.index <= datetime.today())]
    except Exception as e:
        print(f"❌ Failed to parse CSV for {symbol}: {e}")
        continue

    result = simulate_trade(strategies[resolved_name], symbol, df, signal_date)
    results.append({
        "symbol": symbol,
        "strategy": resolved_name,
        "signal_date": signal_date.strftime('%m/%d/%y'),
        "status": result["status"],
        "entry_date": result.get("entry_date", ""),
        "exit_date": result.get("exit_date", "")
    })

results_df = pd.DataFrame(results)
results_df.head()


In [ ]:
# Save results
results_df.to_csv("trade_results.csv", index=False)
print("✅ Results saved to trade_results.csv")
